# tensors
A **tensor** is a generalization of vectors and matrices to potentially higher dimisions. Internally, Tensorsflow represents tensors as n-dimensional arrays of basic datatyeps.

When writing a tensorflow program, the main object you maniplate and pass around is the ```tf.Tensor```. A ```tf.Tensor``` object represnets a partially defined computation that will eventually produce a value.

A ```tf.Tensor``` has the following properties:
- a datatype(float32, int32, or srting)
- a shape

The main tensors are:
- ```tf.Variable```
- ```tf.constant```
- ```tf.placeholder```





## Tensor has two primary purposes

- a tensor can be passed as input to another operation. This build a dataflow connection 
between operations
- After the graph has been launched in a session, the value of tensor can be computed by
passing to tf.Session().run()

In [1]:
import tensorflow as tf

/root/anaconda3/envs/tf1.4_gpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/root/anaconda3/envs/tf1.4_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
tf.reset_default_graph()
# Build a dataflow graph
c = tf.constant([[1.0,2.0],[2.0,4.0]],name='c')
d = tf.constant([[1.0,1.0],[0,1.0]],name='d')
# a tensor can be passed as input to another operation. This build a dataflow connection 
#between operations
e = tf.matmul(c,d,name='e')
#A Tensor is a symbolic handle to one of the outputs of an Operations.
print(e)
print(tf.get_default_graph().as_graph_def())

Tensor("e:0", shape=(2, 2), dtype=float32)
node {
  name: "c"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\200?\000\000\000@\000\000\000@\000\000\200@"
      }
    }
  }
}
node {
  name: "d"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\200?\000\000\200?\000\000\000\000\000\000\200?"
      }
    }
  }
}
node {
  name: "e"
  op: "MatMul"
  input: "c"
  input: "d"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    ke

In [7]:
print(tf.get_default_graph().as_graph_def().node)

[name: "c"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
        dim {
          size: 2
        }
        dim {
          size: 2
        }
      }
      tensor_content: "\000\000\200?\000\000\000@\000\000\000@\000\000\200@"
    }
  }
}
, name: "d"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
        dim {
          size: 2
        }
        dim {
          size: 2
        }
      }
      tensor_content: "\000\000\200?\000\000\200?\000\000\000\000\000\000\200?"
    }
  }
}
, name: "e"
op: "MatMul"
input: "c"
input: "d"
attr {
  key: "T"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "transpose_a"
  value {
    b: false
  }
}
attr {
  key: "transpose_b"
  value {
    b: false
  }
}
]


In [32]:
# After the graph has been launched in a session, the value of tensor can be computed by
# passing to tf.Session().run()
sess = tf.Session()
#Note c,d,e are symbolic tensor objects, whereas result is a numpy array 
#that stores a concret value
result =  sess.run(e)
print result

[[ 1.  3.]
 [ 2.  6.]]


## Understanding static and dynamic shapes

In [16]:
# static shape
a = tf.placeholder(tf.float32,[None,3])
static_shape = a.shape.as_list()
print(static_shape)

[None, 3]


In [17]:
# dynamic shape
dynamic_shape = tf.shape(a) # resturn a Tensor
print(dynamic_shape)


Tensor("Shape_4:0", shape=(2,), dtype=int32)


In [18]:
# print dynamic shape inside Session
dynamic_shape = tf.shape(a) # resturn a Tensor
with tf.Session() as sess:
    cout = sess.run(dynamic_shape, feed_dict={a:[[1,2,3], [4,5,6]]})
    print(cout)

[2 3]


In [19]:
sess=tf.Session()
cout = sess.run(dynamic_shape, feed_dict={a:[[1,2,3], [4,5,6]]})
print(cout)
sess.close()

[2 3]


In [22]:
b = tf.placeholder(tf.float32, [None, 10, 32])
b_static_shape_list = b.shape
print(b_static_shape_list)

(?, 10, 32)


In [25]:
b_static_shape_list[0], b_static_shape_list[1], b_static_shape_list[2]

(Dimension(None), Dimension(10), Dimension(32))

In [23]:
b_reshape = tf.reshape(b,[b_static_shape_list[0],
                          b_static_shape_list[1]*b_static_shape_list[2]])

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [Dimension(None), Dimension(320)]. Consider casting elements to a supported type.

In [26]:

def get_shape(tensor):
    static_shape = tensor.shape.as_list()
    dynamic_shape =  tf.unstack(tf.shape(tensor))
    # 若static_shape的某一維為None﹐則取dynamic_shape的一維
    dims = [s[1] if s[0] is None else s[0] for s in zip(static_shape,dynamic_shape)]
    return dims

shape = get_shape(b)
print(shape)
b = tf.reshape(b,[shape[0],shape[1]*shape[2]])
print(b)

[<tf.Tensor 'unstack_3:0' shape=() dtype=int32>, 10, 32]
Tensor("Reshape_6:0", shape=(?, 320), dtype=float32)
